<a href="https://colab.research.google.com/github/MSaadShahid1/ScholarMind/blob/main/ITSOLERA_PROJECT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate


In [ ]:
import json
import requests
import torch # Still useful for GPU detection, though not directly for GPT-2 generation now
from transformers import pipeline # Keep for potential local tokenizer use or if switching back


In [ ]:
# Determine if a GPU is available and set the device accordingly
device = 0 if torch.cuda.is_available() else -1

print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

# This generator is kept for demonstration purposes if you wish to use a local model.
# For the main paper generation, we will be using an API call to a more powerful LLM.
# generator = pipeline('text-generation', model='gpt2', device=device)

# print("Local GPT-2 Model loaded successfully (if uncommented)!")


Using device: CPU


In [ ]:
def generate_text_snippet_with_llm_api(prompt, max_length=2000, temperature=0.7):
    """
    Generates a single text snippet based on a given prompt using the Gemini 2.0 Flash API.

    Args:
        prompt (str): The initial text prompt from the user.
        max_length (int): The maximum total length of the generated text (approximate tokens).
                          This is passed as part of the generationConfig.
        temperature (float): Controls the randomness of the generation.

    Returns:
        str: The generated text snippet.
    """
    print(f"\nGenerating text using LLM API for prompt: '{prompt}'...")

    chatHistory = []
    # Corrected: Use .append() for Python lists instead of .push()
    chatHistory.append({ "role": "user", "parts": [{ "text": prompt }] })

    payload = {
        "contents": chatHistory,
        "generationConfig": {
            "maxOutputTokens": max_length,
            "temperature": temperature,
            # top_k and top_p are not directly exposed in this simplified API call,
            # but temperature helps control randomness.
        }
    }
    # Your API key is inserted here to resolve the 403 error.
    # In a production environment, use environment variables for security.
    apiKey = # Add your Key in ""
    apiUrl = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={apiKey}"

    try:
        # Simulate fetch with requests.post for Colab environment
        response = requests.post(apiUrl, headers={'Content-Type': 'application/json'}, data=json.dumps(payload))
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()

        if result.get("candidates") and len(result["candidates"]) > 0 and \
           result["candidates"][0].get("content") and result["candidates"][0]["content"].get("parts") and \
           len(result["candidates"][0]["content"]["parts"]) > 0:
            text = result["candidates"][0]["content"]["parts"][0]["text"]
            return text
        else:
            print("LLM API response structure unexpected or content missing.")
            # Print the full result for debugging
            print(f"Full API response: {result}")
            return "Error: Could not generate text from LLM API."
    except requests.exceptions.RequestException as e:
        print(f"Error calling LLM API: {e}")
        return f"Error: Failed to connect to LLM API. {e}"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"Error: An unexpected error occurred during API call. {e}"


def generate_full_paper(main_topic, max_section_length=2000):
    """
    Generates a full paper with structured sections based on a main topic using the LLM API.

    Args:
        main_topic (str): The central topic or research question for the paper.
        max_section_length (int): The maximum length (in tokens) for each individual section.
                                  Note: Generating a paper of 20,000+ words in a single run
                                  with any LLM is challenging due to context window limitations,
                                  quality degradation over extremely long generations, and
                                  computational constraints. This setting increases section length
                                  but a full 20,000-word paper would require more advanced techniques
                                  (e.g., iterative generation, retrieval-augmented generation,
                                  and careful post-processing).

    Returns:
        str: The complete generated paper.
    """
    print(f"\n--- Generating Full Paper on: '{main_topic}' ---")
    full_paper_content = []

    # Abstract
    abstract_prompt = f"Write a concise abstract for a research paper on '{main_topic}', summarizing its purpose, methods, key findings, and conclusions."
    abstract = generate_text_snippet_with_llm_api(abstract_prompt, max_length=max_section_length)
    full_paper_content.append("## Abstract\n" + abstract.replace(abstract_prompt, "").strip()) # Remove prompt from output

    # Introduction
    intro_prompt = f"Write an introduction for a research paper on '{main_topic}', outlining the background, problem statement, and the paper's objectives."
    introduction = generate_text_snippet_with_llm_api(intro_prompt, max_length=max_section_length)
    full_paper_content.append("\n## 1. Introduction\n" + introduction.replace(intro_prompt, "").strip())

    # Literature Review
    lit_review_prompt = f"Conduct a literature review for a research paper on '{main_topic}', discussing existing research, theories, and identifying gaps."
    literature_review = generate_text_snippet_with_llm_api(lit_review_prompt, max_length=max_section_length)
    full_paper_content.append("\n## 2. Literature Review\n" + literature_review.replace(lit_review_prompt, "").strip())

    # Methodology
    methodology_prompt = f"Describe the methodology for a research paper on '{main_topic}', including research design, participants/data, and data collection/analysis methods."
    methodology = generate_text_snippet_with_llm_api(methodology_prompt, max_length=max_section_length)
    full_paper_content.append("\n## 3. Methodology\n" + methodology.replace(methodology_prompt, "").strip())

    # Results (Hypothetical)
    results_prompt = f"Present the hypothetical results for a research paper on '{main_topic}', describing key findings and observations."
    results = generate_text_snippet_with_llm_api(results_prompt, max_length=max_section_length)
    full_paper_content.append("\n## 4. Results\n" + results.replace(results_prompt, "").strip())

    # Conclusion
    conclusion_prompt = f"Write a conclusion for a research paper on '{main_topic}', summarizing the main findings, their implications, and suggesting future research directions."
    conclusion = generate_text_snippet_with_llm_api(conclusion_prompt, max_length=max_section_length)
    full_paper_content.append("\n## 5. Conclusion\n" + conclusion.replace(conclusion_prompt, "").strip())

    return "\n".join(full_paper_content)


In [ ]:
from IPython.display import HTML, display

# IMPORTANT: Replace "YOUR_API_KEY_HERE" with your actual Google Generative AI API key.
# This is necessary because the Colab environment does not automatically inject it
# into the HTML/JavaScript when rendered via IPython.display.HTML.
api_key_for_colab = #Add you key in ""

html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ScholarMind - AI Paper Generator</title>
    <!-- Tailwind CSS CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        /* Custom font for a clean look */
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
        body {{
            font-family: 'Inter', sans-serif;
            background-color: #e0f2fe; /* Lighter blue background */
            background-image: linear-gradient(to bottom right, #e0f2fe, #bbdefb); /* Subtle gradient */
        }}
        .container {{
            max-width: 900px;
            margin: 2rem auto;
            padding: 2.5rem; /* Increased padding */
            background-color: #ffffff;
            border-radius: 1.5rem; /* More rounded corners */
            box-shadow: 0 15px 40px rgba(0, 0, 0, 0.15); /* Stronger shadow */
            transition: all 0.4s ease-in-out;
            border: 1px solid #cceeff; /* Light border */
        }}
        .container:hover {{
            box-shadow: 0 20px 50px rgba(0, 0, 0, 0.2);
            transform: translateY(-5px); /* Subtle lift effect */
        }}
        textarea {{
            resize: vertical;
            min-height: 80px; /* Min height for textarea */
        }}
        /* Custom scrollbar for generated content */
        .scrollable-content::-webkit-scrollbar {{
            width: 8px;
        }}
        .scrollable-content::-webkit-scrollbar-track {{
            background: #f1f1f1;
            border-radius: 10px;
        }}
        .scrollable-content::-webkit-scrollbar-thumb {{
            background: #90caf9; /* Light blue scrollbar */
            border-radius: 10px;
        }}
        .scrollable-content::-webkit-scrollbar-thumb:hover {{
            background: #64b5f6; /* Darker blue on hover */
        }}
        /* Button gradient and hover effect */
        .btn-primary {{
            background-image: linear-gradient(to right, #42a5f5, #2196f3);
            transition: all 0.3s ease;
        }}
        .btn-primary:hover {{
            background-image: linear-gradient(to right, #2196f3, #1976d2);
            box-shadow: 0 4px 15px rgba(33, 150, 243, 0.4);
        }}
        .btn-secondary {{
            background-image: linear-gradient(to right, #66bb6a, #43a047);
            transition: all 0.3s ease;
        }}
        .btn-secondary:hover {{
            background-image: linear-gradient(to right, #43a047, #2e7d32);
            box-shadow: 0 4px 15px rgba(76, 175, 80, 0.4);
        }}
    </style>
</head>
<body class="bg-gray-100 min-h-screen flex items-center justify-center p-4 sm:p-6 lg:p-8">

    <div class="container w-full">
        <h1 class="text-5xl font-extrabold text-center text-gray-900 mb-4 tracking-tight">ScholarMind</h1>
        <p class="text-center text-blue-700 text-xl font-medium mb-10">Your AI Co-Author for Academic Research</p>

        <!-- Input Section -->
        <div class="mb-10 p-7 bg-blue-50 rounded-xl shadow-lg border border-blue-200">
            <label for="topicInput" class="block text-xl font-semibold text-gray-800 mb-4">Enter your paper topic:</label>
            <textarea id="topicInput" rows="4" class="w-full p-5 border border-blue-400 rounded-xl focus:ring-blue-600 focus:border-blue-600 transition duration-300 ease-in-out text-gray-800 placeholder-gray-500 shadow-sm text-lg" placeholder="e.g., The ethical implications of AI in healthcare"></textarea>
            <button id="generateBtn" class="mt-6 w-full btn-primary text-white py-4 px-8 rounded-xl font-bold text-xl hover:shadow-xl focus:outline-none focus:ring-4 focus:ring-blue-300 transition duration-300 ease-in-out transform hover:scale-105 shadow-lg">
                Generate Paper
            </button>
        </div>

        <!-- Loading Indicator -->
        <div id="loadingIndicator" class="hidden text-center text-blue-600 font-medium text-xl mb-10">
            <div class="flex items-center justify-center">
                <svg class="animate-spin -ml-1 mr-4 h-10 w-10 text-blue-600" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                    <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                    <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                Generating your comprehensive paper... This may take a few moments.
            </div>
        </div>

        <!-- Generated Paper Output -->
        <div id="outputSection" class="hidden p-7 bg-gray-50 rounded-xl shadow-xl border border-gray-200">
            <h2 class="text-3xl font-bold text-gray-800 mb-5">Generated Paper:</h2>
            <div id="paperOutput" class="scrollable-content h-96 overflow-y-auto bg-white p-6 rounded-lg border border-gray-300 text-gray-700 leading-relaxed whitespace-pre-wrap text-base sm:text-lg">
                <!-- Generated content will appear here -->
            </div>
            <button id="copyBtn" class="mt-6 w-full btn-secondary text-white py-3 px-6 rounded-xl font-bold text-lg hover:shadow-xl focus:outline-none focus:ring-4 focus:ring-green-300 transition duration-300 ease-in-out transform hover:scale-105 shadow-lg">
                Copy Paper
            </button>
        </div>

        <!-- Message Box for Alerts (instead of alert()) -->
        <div id="messageBox" class="hidden fixed inset-0 bg-gray-600 bg-opacity-60 flex items-center justify-center p-4 z-50">
            <div class="bg-white rounded-lg p-7 shadow-2xl max-w-sm w-full text-center">
                <p id="messageText" class="text-gray-800 text-xl mb-5 font-semibold"></p>
                <button id="messageCloseBtn" class="bg-blue-600 text-white py-2.5 px-6 rounded-lg hover:bg-blue-700 transition duration-200 text-lg font-medium">OK</button>
            </div>
        </div>

    </div>

    <script>
        const topicInput = document.getElementById('topicInput');
        const generateBtn = document.getElementById('generateBtn');
        const loadingIndicator = document.getElementById('loadingIndicator');
        const outputSection = document.getElementById('outputSection');
        const paperOutput = document.getElementById('paperOutput');
        const copyBtn = document.getElementById('copyBtn');
        const messageBox = document.getElementById('messageBox');
        const messageText = document.getElementById('messageText');
        const messageCloseBtn = document.getElementById('messageCloseBtn');

        // Function to show custom message box
        function showMessage(message) {{
            messageText.textContent = message;
            messageBox.classList.remove('hidden');
        }}

        // Function to hide custom message box
        messageCloseBtn.addEventListener('click', () => {{
            messageBox.classList.add('hidden');
        }});

        // IMPORTANT: Your Google Generative AI API key.
        // This is necessary when running in Google Colab via IPython.display.HTML.
        // In the Canvas environment, this would be automatically provided.
        const apiKey = "{api_key_for_colab}";

        // Simulates the Python backend's generate_text_snippet_with_llm_api function
        async function generateTextSnippet(prompt, maxLength = 2000, temperature = 0.7) {{
            console.log(`Generating text for prompt: '${{prompt}}' using LLM API...`);

            const chatHistory = [];
            chatHistory.push({{ role: "user", parts: [{{ text: prompt }}] }});

            const payload = {{
                contents: chatHistory,
                generationConfig: {{
                    maxOutputTokens: maxLength,
                    temperature: temperature,
                }}
            }};

            const apiUrl = `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key=${{apiKey}}`;

            try {{
                const response = await fetch(apiUrl, {{
                    method: 'POST',
                    headers: {{ 'Content-Type': 'application/json' }},
                    body: JSON.stringify(payload)
                }});

                if (!response.ok) {{
                    const errorData = await response.json();
                    console.error("LLM API Error:", errorData);
                    throw new Error(`HTTP error! status: ${{response.status}} - ${{errorData.error.message || 'Unknown error'}}`);
                }}

                const result = await response.json();

                if (result.candidates && result.candidates.length > 0 &&
                    result.candidates[0].content && result.candidates[0].content.parts &&
                    result.candidates[0].content.parts.length > 0) {{
                    const text = result.candidates[0].content.parts[0].text;
                    return text;
                }} else {{
                    console.error("LLM API response structure unexpected or content missing:", result);
                    return "Error: Could not generate text from LLM API. Unexpected response.";
                }}
            }} catch (e) {{
                console.error("Error calling LLM API:", e);
                return `Error: Failed to connect to LLM API. ${{e.message}}`;
            }}
        }}

        // Simulates the Python backend's generate_full_paper function
        async function generateFullPaper(mainTopic, maxSectionLength = 2000) {{
            let fullPaperContent = [];

            const sections = [
                {{ title: "Abstract", prompt: (topic) => `Write a concise abstract for a research paper on '${{topic}}', summarizing its purpose, methods, key findings, and conclusions.` }},
                {{ title: "1. Introduction", prompt: (topic) => `Write an an in-depth introduction for a research paper on '${{topic}}', outlining the background, a detailed problem statement, and the paper's objectives. Ensure it flows logically and sets the stage for the research.` }},
                {{ title: "2. Literature Review", prompt: (topic) => `Conduct a comprehensive literature review for a research paper on '${{topic}}', discussing key existing research, relevant theories, and clearly identifying gaps in current knowledge. Aim for detailed analysis.` }},
                {{ title: "3. Methodology", prompt: (topic) => `Describe a robust methodology for a research paper on '${{topic}}', including a detailed research design (e.g., qualitative, quantitative, mixed-methods), proposed participants/data sources, and specific data collection and analysis methods. Be thorough.` }},
                {{ title: "4. Results", prompt: (topic) => `Present detailed hypothetical results for a research paper on '${{topic}}', describing key findings, observations, and potential statistical outcomes. Structure it with sub-sections if appropriate.` }},
                {{ title: "5. Discussion", prompt: (topic) => `Provide a comprehensive discussion section for a research paper on '${{topic}}', interpreting the hypothetical results in the context of the literature review, discussing implications, limitations, and potential future research directions.`}},
                {{ title: "6. Conclusion", prompt: (topic) => `Write a strong conclusion for a research paper on '${{topic}}', summarizing the main findings, reiterating their significance, and offering a final thought or call to action.` }},
                {{ title: "7. References", prompt: (topic) => `Generate a list of 10-15 hypothetical academic references (books, journal articles, conference papers) related to a research paper on '${{topic}}'. Format them in a consistent style (e.g., APA or MLA). IMPORTANT: These are illustrative examples and should not be considered real, verifiable sources.`}}
            ];

            for (const section of sections) {{
                const sectionPrompt = section.prompt(mainTopic);
                let generatedText = await generateTextSnippet(sectionPrompt, maxSectionLength);

                // Attempt to remove the prompt from the generated text if it's present at the beginning
                if (generatedText.startsWith(sectionPrompt)) {{
                    generatedText = generatedText.substring(sectionPrompt.length).trim();
                }}
                fullPaperContent.push(`## ${{section.title}}\\n${{generatedText}}`);
            }}

            return fullPaperContent.join('\\n\\n'); // Add double newline for better section separation
        }}

        // Event listener for the Generate button
        generateBtn.addEventListener('click', async () => {{
            const topic = topicInput.value.trim();

            if (!topic) {{
                showMessage("Please enter a paper topic before generating.");
                return;
            }}

            // Show loading indicator and hide output
            loadingIndicator.classList.remove('hidden');
            outputSection.classList.add('hidden');
            paperOutput.textContent = ''; // Clear previous content

            try {{
                const generatedPaper = await generateFullPaper(topic);
                paperOutput.textContent = generatedPaper;
                outputSection.classList.remove('hidden');
            }} catch (error) {{
                console.error("Error generating full paper:", error);
                showMessage(`Failed to generate paper: ${{error.message}}. Please try again.`);
            }} finally {{
                // Hide loading indicator
                loadingIndicator.classList.add('hidden');
            }}
        }});

        // Event listener for the Copy button
        copyBtn.addEventListener('click', () => {{
            const textToCopy = paperOutput.textContent;
            if (textToCopy) {{
                // Use document.execCommand('copy') for clipboard operations in iframes
                const textarea = document.createElement('textarea');
                textarea.value = textToCopy;
                document.body.appendChild(textarea);
                textarea.select();
                try {{
                    document.execCommand('copy');
                    showMessage('Paper copied to clipboard!');
                }} catch (err) {{
                    console.error('Failed to copy text: ', err);
                    showMessage('Failed to copy paper to clipboard. Please copy manually.');
                }}
                document.body.removeChild(textarea);
            }} else {{
                showMessage('No content to copy!');
            }}
        }});
    </script>
</body>
</html>
"""

display(HTML(html_content))
